In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
base = pd.read_csv("BASE.txt",sep=";")
monedas = pd.read_csv("Monedas.txt", sep=";")

In [3]:
base = base.replace({'Empresa': {'EmpresaC':'Empresa C','Empesa B':'Empresa B'}})

In [4]:
base.rename(columns = {'Fecha':'Date'}, inplace = True) 
monedas.rename(columns = {'FECHA':'Date'}, inplace = True)
print(base.shape)
monedas.shape

(3383, 4)


(365, 3)

In [5]:
base.columns

Index(['Date', 'Empresa', 'Moneda', 'Ingreso'], dtype='object')

In [6]:
df_unido = pd.merge(base, monedas, on = 'Date', how='inner',left_index=True)
df_unido.shape

(3383, 6)

In [7]:
df_unido.dtypes

Date        object
Empresa     object
Moneda      object
Ingreso    float64
USD        float64
EURO       float64
dtype: object

In [8]:
df_unido['year'] = pd.to_datetime(df_unido['Date']).dt.year
df_unido['trim'] = pd.to_datetime(df_unido['Date']).dt.quarter

In [9]:
df_unido.dtypes

Date        object
Empresa     object
Moneda      object
Ingreso    float64
USD        float64
EURO       float64
year         int64
trim         int64
dtype: object

In [10]:
df_unido.head()

,Date,Empresa,Moneda,Ingreso,USD,EURO,year,trim
0,01/01/2017,Empresa A,EURO,0.000137,3000.71,3136.64,2017,1
0,01/01/2017,Empresa B,EURO,0.000329,3000.71,3136.64,2017,1
0,01/01/2017,Empresa C,EURO,0.000082,3000.71,3136.64,2017,1
0,01/01/2017,Empresa A,PESO,183.766913,3000.71,3136.64,2017,1
0,01/01/2017,Empresa B,PESO,871.384057,3000.71,3136.64,2017,1


In [12]:
df_unido['fecha1'] = pd.to_datetime(df_unido['Date'], format='%d/%m/%Y')
df_unido.dtypes

Date               object
Empresa            object
Moneda             object
Ingreso           float64
USD               float64
EURO              float64
year                int64
trim                int64
fecha1     datetime64[ns]
dtype: object

In [13]:
df_unido['trim1'] = pd.to_datetime(df_unido['fecha1']).dt.quarter

In [16]:
df_unido.loc[df_unido['Date']=='01/04/2017']

,Date,Empresa,Moneda,Ingreso,USD,EURO,year,trim,fecha1,trim1
90,01/04/2017,Empresa A,EURO,0.000118,2885.57,3084.96,2017,1,2017-04-01,2
90,01/04/2017,Empresa B,EURO,0.000118,2885.57,3084.96,2017,1,2017-04-01,2
90,01/04/2017,Empresa C,EURO,0.000071,2885.57,3084.96,2017,1,2017-04-01,2
90,01/04/2017,Empresa A,PESO,174.953474,2885.57,3084.96,2017,1,2017-04-01,2
90,01/04/2017,Empresa B,PESO,828.180246,2885.57,3084.96,2017,1,2017-04-01,2
90,01/04/2017,Empresa C,PESO,342.791651,2885.57,3084.96,2017,1,2017-04-01,2
90,01/04/2017,Empresa A,USD,0.015233,2885.57,3084.96,2017,1,2017-04-01,2
90,01/04/2017,Empresa B,USD,0.020325,2885.57,3084.96,2017,1,2017-04-01,2
90,01/04/2017,Empresa C,USD,0.007519,2885.57,3084.96,2017,1,2017-04-01,2


In [17]:
df = df_unido
df.reset_index(inplace=True)
Ingreso_t=[]
for i in range(len(df)):
    mon = df['Moneda'][i]
    ingreso = df['Ingreso'][i]
    usd = df['USD'][i]
    euro = df['EURO'][i]
    if mon=='USD':
        ing = ingreso*usd
    elif mon=='EURO':
        ing = ingreso*euro
    else:
        ing = ingreso
    Ingreso_t.append(ing)

df['Ingreso_Total'] = Ingreso_t


In [18]:
pd.options.display.float_format="{:,.2f}".format
ing_trim_pesos = df.groupby(['Empresa','trim1']).agg({'Ingreso_Total':['sum']})
ing_trim_pesos.columns =['Ingreso_Total_pesos']
ing_trim_pesos

Ingreso_Total_pesos
Empresa   trim1                     
Empresa A 1                39,602.50
          2                15,923.16
          3                24,539.74
          4                21,610.99
Empresa B 1               118,815.80
          2                81,849.51
          3                82,643.15
          4                92,439.95
Empresa C 1                50,244.52
          2                36,663.35
          3                31,680.54
          4                43,162.27

In [28]:
#df_sem = df.loc[df['fecha1']<='2017-06-30']
df_sem = df.loc[(df['fecha1']>='2017-01-01')&(df['fecha1']<='2017-06-30')]
df_sem.shape
ing_sem_pesos = df_sem.groupby(['Empresa']).agg({'Ingreso_Total':['sum','count']})
ing_sem_pesos.columns =['Ingreso_pesos', 'Cantidad_trx']
ing_sem_pesos


,Ingreso_pesos,Cantidad_trx
Empresa,,
Empresa A,"55,525.66",543
Empresa B,"200,665.32",543
Empresa C,"86,907.87",573
